#### 2010美國各州地區人口密度排行

In [16]:
# Python 3
# 先取得需要的資料
import urllib.request
for i in ["abbrevs", "areas", "population"]:
    url = "https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-"+i+".csv"
    # remenber url choose raw-link over web one, or you will get platform html info.
    filename = urllib.request.urlretrieve(url, filename="./data/"+i+".csv")
    # 位置放在本地端/data
    print("download file ",i)


download file  abbrevs
download file  areas
download file  population


In [20]:
import numpy as np
import pandas as pd

In [19]:
# 看一下資料
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

print(pop.head()); print(areas.head()); print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [21]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [22]:
# drop duplicate info.
merged = merged.drop('abbreviation', 1)
merged.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15696\1746873960.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('abbreviation', 1)


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [24]:
# 接下來確認欄位是否有空值
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [26]:
popNull = merged['population'].isnull()
merged[popNull].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [29]:
print(merged.loc[merged['population'].isnull(), 'year'].unique())
print(merged.loc[merged['population'].isnull(), 'state/region'].unique())

[1990 1991 1993 1992 1994 1995 1996 1998 1997 1999]
['PR']


#### 在pop上的缺失是由PR 1990年代未記錄造成的

In [30]:
# 看一下state的缺失怎麼來的?
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

#### state各州全稱上未加入PR, USA

In [31]:
# get them in!
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

#### 因為PR19s 有資料缺失，做人口密度就直接去掉那一年代前的

In [32]:
# finally, let area square in
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [33]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [34]:
# area出現缺失 看一下
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

In [18]:
# 想想看，我們要算各州人口密度。US是整個土地的總面積，不去計算這個值也可以，那就先放掉不管。
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [37]:
# 開始算人口密度排名
data2012 = final.query("year == 2012 & ages == 'total'")
data2012.head()

,state/region,ages,year,population,state,area (sq. mi)
1,AL,total,2012,4817528.0,Alabama,52423.0
95,AK,total,2012,730307.0,Alaska,656425.0
97,AZ,total,2012,6551149.0,Arizona,114006.0
191,AR,total,2012,2949828.0,Arkansas,53182.0
193,CA,total,2012,37999878.0,California,163707.0


In [38]:
data2012.set_index('state', inplace=True)
density = data2012['population'] / data2012['area (sq. mi)']

In [39]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    9315.102941
Puerto Rico             1038.846373
New Jersey              1016.710502
Rhode Island             679.808414
Connecticut              647.865260
dtype: float64

In [40]:
density.tail()

state
North Dakota     9.919453
Montana          6.837955
Wyoming          5.894886
Alaska           1.112552
United States         NaN
dtype: float64